In [149]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

### Import Dataset

In [150]:
data = pd.read_csv('student-mat.csv',sep = ";")

### Show first 20 data

In [151]:
data.head(20)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
5,GP,M,16,U,LE3,T,4,3,services,other,...,5,4,2,1,2,5,10,15,15,15
6,GP,M,16,U,LE3,T,2,2,other,other,...,4,4,4,1,1,3,0,12,12,11
7,GP,F,17,U,GT3,A,4,4,other,teacher,...,4,1,4,1,1,1,6,6,5,6
8,GP,M,15,U,LE3,A,3,2,services,other,...,4,2,2,1,1,1,0,16,18,19
9,GP,M,15,U,GT3,T,3,4,other,other,...,5,5,1,1,1,5,0,14,15,15


In [152]:
from sklearn.preprocessing import LabelEncoder

In [153]:
le_studytime = LabelEncoder()
le_failures = LabelEncoder()
le_schoolsup = LabelEncoder()
le_famsup = LabelEncoder()
le_paid = LabelEncoder()
le_higher = LabelEncoder()
le_internet = LabelEncoder()
le_health = LabelEncoder()
le_absences = LabelEncoder()

In [154]:
data['studytime_n'] = le_studytime.fit_transform(data['studytime'])
data['failures_n'] = le_failures.fit_transform(data['failures'])
data['schoolsup_n'] = le_schoolsup.fit_transform(data['schoolsup'])
data['famsup_n'] = le_studytime.fit_transform(data['famsup'])
data['paid_n'] = le_failures.fit_transform(data['paid'])
data['higher_n'] = le_schoolsup.fit_transform(data['higher'])
data['internet_n'] = le_studytime.fit_transform(data['internet'])
data['health_n'] = le_failures.fit_transform(data['health'])
data['absences_n'] = le_schoolsup.fit_transform(data['absences'])
data.head(10)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,G3,studytime_n,failures_n,schoolsup_n,famsup_n,paid_n,higher_n,internet_n,health_n,absences_n
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,6,1,0,1,0,0,1,0,2,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,6,1,0,0,1,0,1,1,2,4
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,10,1,3,1,0,1,1,1,2,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,15,2,0,0,1,1,1,1,4,2
4,GP,F,16,U,GT3,T,3,3,other,other,...,10,1,0,0,1,1,1,0,4,4
5,GP,M,16,U,LE3,T,4,3,services,other,...,15,1,0,0,1,1,1,1,4,10
6,GP,M,16,U,LE3,T,2,2,other,other,...,11,1,0,0,0,0,1,1,2,0
7,GP,F,17,U,GT3,A,4,4,other,teacher,...,6,1,0,1,1,0,1,0,0,6
8,GP,M,15,U,LE3,A,3,2,services,other,...,19,1,0,0,1,1,1,1,0,0
9,GP,M,15,U,GT3,T,3,4,other,other,...,15,1,0,0,1,1,1,1,4,0


In [155]:
from pandas import DataFrame

In [156]:
x_data = data[['studytime_n','failures_n','schoolsup_n','famsup_n','paid_n','internet_n','health_n','absences_n']]
y_data =data[['higher_n']]

In [157]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data, test_size=0.2, random_state=3)

In [158]:
from sklearn import linear_model

In [159]:
y_train

,higher_n
268,1
236,1
23,1
247,0
112,1
...,...
256,1
131,1
249,1
152,1


In [160]:
reg = linear_model.LogisticRegression()
reg.fit(x_train, y_train.values.ravel())

LogisticRegression()

In [161]:
reg.predict([[1,0,1,0,0,0,2,6]])

array([1])